In [174]:
%reload_ext autoreload
%autoreload 1
import duckdb
import pandas as pd
from IPython.display import Markdown, display
from tabulate import tabulate
import matplotlib.pyplot as plt
import numpy as np
import functions.db_functions as dbf
%aimport functions.db_functions

In [175]:
con = duckdb.connect()
con.execute(f"INSTALL sqlite")
con.execute(f"LOAD sqlite")
con.execute(f"SET GLOBAL sqlite_all_varchar=true")
con.execute(f"CALL sqlite_attach('data/database.sqlite')")

#### What are the contents of the dataset?

##### Getting table names and column counts:

In [176]:
tables = con.query(
    f"""--sql
SELECT table_name as 'Table Name',
    COUNT(column_name) AS 'Column Count'
FROM information_schema.columns
GROUP BY table_name
"""
).to_df()

##### Getting the column names:

In [177]:
table_columns = con.query(
    f"""--sql
SELECT table_name,
    GROUP_CONCAT(column_name) AS column_names
FROM information_schema.columns
GROUP BY table_name"""
).to_df()
tables["column_names"] = table_columns["column_names"].apply(lambda x: x.split(","))

##### Getting the row count:

In [178]:
row_counts = {}
for table_name in tables["Table Name"]:
    row_counts[table_name] = con.query(
        f"""--sql
    SELECT COUNT(*) FROM {table_name}
        """
    ).fetchone()[0]

tables = tables.join(pd.Series(row_counts).rename("Row Count"), on="Table Name")

##### Dataset contents:

In [179]:
Markdown(tabulate(
tables[['Table Name','Column Count','Row Count']]
, showindex=False, headers="keys", tablefmt="pipe"))

| Table Name        |   Column Count |   Row Count |
|:------------------|---------------:|------------:|
| sqlite_sequence   |              2 |           7 |
| Player_Attributes |             42 |      183978 |
| Player            |              7 |       11060 |
| Match             |            115 |       25979 |
| League            |              3 |          11 |
| Country           |              2 |          11 |
| Team              |              5 |         299 |
| Team_Attributes   |             25 |        1458 |

##### Exporting the column names of each table into an xlsx file for reference:

In [180]:
# tables_writer=pd.ExcelWriter('tables.xlsx')
# for i, tab in enumerate(tables.explode('column_names').groupby('table_name')):
#     df_to_write=tab[1]
#     df_to_write.rename(columns={'column_names':tab[0]},inplace=True)
#     df_to_write.to_excel(tables_writer,columns=[tab[0]],startcol=i,index=False)
# tables_writer.close()

### Player attributes

#### How many players are there in the player table?

In [181]:
con.query(
    """--sql
SELECT count(distinct player_api_id) FROM Player"""
).fetchone()[0]

11060

##### What is the data in the player table?

In [182]:
Markdown(tabulate(con.query(
    """--sql
SELECT * from Player LIMIT 5"""
).to_df(), showindex=False, headers="keys", tablefmt="pipe"))

|   id |   player_api_id | player_name        |   player_fifa_api_id | birthday            |   height |   weight |
|-----:|----------------:|:-------------------|---------------------:|:--------------------|---------:|---------:|
|    1 |          505942 | Aaron Appindangoye |               218353 | 1992-02-29 00:00:00 |   182.88 |      187 |
|    2 |          155782 | Aaron Cresswell    |               189615 | 1989-12-15 00:00:00 |   170.18 |      146 |
|    3 |          162549 | Aaron Doran        |               186170 | 1991-05-13 00:00:00 |   170.18 |      163 |
|    4 |           30572 | Aaron Galindo      |               140161 | 1982-05-08 00:00:00 |   182.88 |      198 |
|    5 |           23780 | Aaron Hughes       |                17725 | 1979-11-08 00:00:00 |   182.88 |      154 |

##### Are there null values in the player table?

In [183]:
dbf.check_db_nulls(con, "Player")

No nulls in id Column

No nulls in player_api_id Column

No nulls in player_name Column

No nulls in player_fifa_api_id Column

No nulls in birthday Column

No nulls in height Column

No nulls in weight Column

##### How many entries are in the player attributes table?

In [184]:
con.query(
    """--sql
SELECT count(player_api_id)
FROM Player_Attributes
WHERE player_api_id IN (SELECT player_api_id FROM Player)
    """
).fetchone()[0]

183978

##### Do all player have entries in the attributes table?

In [185]:
dbf.check_db_refs(con, "Player", "Player_Attributes", "player_api_id")

All entries in Player are referenced in Player_Attributes.

#### What are the contents of Player attributes table?

In [186]:
Markdown(tabulate(con.query(
    """--sql
SELECT * from Player_Attributes LIMIT 5"""
).to_df(), showindex=False, headers="keys", tablefmt="pipe"))

|   id |   player_fifa_api_id |   player_api_id | date                |   overall_rating |   potential | preferred_foot   | attacking_work_rate   | defensive_work_rate   |   crossing |   finishing |   heading_accuracy |   short_passing |   volleys |   dribbling |   curve |   free_kick_accuracy |   long_passing |   ball_control |   acceleration |   sprint_speed |   agility |   reactions |   balance |   shot_power |   jumping |   stamina |   strength |   long_shots |   aggression |   interceptions |   positioning |   vision |   penalties |   marking |   standing_tackle |   sliding_tackle |   gk_diving |   gk_handling |   gk_kicking |   gk_positioning |   gk_reflexes |
|-----:|---------------------:|----------------:|:--------------------|-----------------:|------------:|:-----------------|:----------------------|:----------------------|-----------:|------------:|-------------------:|----------------:|----------:|------------:|--------:|---------------------:|---------------:|---------------:|---------------:|---------------:|----------:|------------:|----------:|-------------:|----------:|----------:|-----------:|-------------:|-------------:|----------------:|--------------:|---------:|------------:|----------:|------------------:|-----------------:|------------:|--------------:|-------------:|-----------------:|--------------:|
|    1 |               218353 |          505942 | 2016-02-18 00:00:00 |               67 |          71 | right            | medium                | medium                |         49 |          44 |                 71 |              61 |        44 |          51 |      45 |                   39 |             64 |             49 |             60 |             64 |        59 |          47 |        65 |           55 |        58 |        54 |         76 |           35 |           71 |              70 |            45 |       54 |          48 |        65 |                69 |               69 |           6 |            11 |           10 |                8 |             8 |
|    2 |               218353 |          505942 | 2015-11-19 00:00:00 |               67 |          71 | right            | medium                | medium                |         49 |          44 |                 71 |              61 |        44 |          51 |      45 |                   39 |             64 |             49 |             60 |             64 |        59 |          47 |        65 |           55 |        58 |        54 |         76 |           35 |           71 |              70 |            45 |       54 |          48 |        65 |                69 |               69 |           6 |            11 |           10 |                8 |             8 |
|    3 |               218353 |          505942 | 2015-09-21 00:00:00 |               62 |          66 | right            | medium                | medium                |         49 |          44 |                 71 |              61 |        44 |          51 |      45 |                   39 |             64 |             49 |             60 |             64 |        59 |          47 |        65 |           55 |        58 |        54 |         76 |           35 |           63 |              41 |            45 |       54 |          48 |        65 |                66 |               69 |           6 |            11 |           10 |                8 |             8 |
|    4 |               218353 |          505942 | 2015-03-20 00:00:00 |               61 |          65 | right            | medium                | medium                |         48 |          43 |                 70 |              60 |        43 |          50 |      44 |                   38 |             63 |             48 |             60 |             64 |        59 |          46 |        65 |           54 |        58 |        54 |         76 |           34 |           62 |              40 |            44 |       53 |          47 |        62 |                63 |               66 |           5 |            10 |            9 |                7 |             7 |
|    5 |               218353 |          505942 | 2007-02-22 00:00:00 |               61 |          65 | right            | medium                | medium                |         48 |          43 |                 70 |              60 |        43 |          50 |      44 |                   38 |             63 |             48 |             60 |             64 |        59 |          46 |        65 |           54 |        58 |        54 |         76 |           34 |           62 |              40 |            44 |       53 |          47 |        62 |                63 |               66 |           5 |            10 |            9 |                7 |             7 |

#### How many different entries can a player have?

In [187]:
con.query(
    f"""--sql
SELECT MAX(num_entries) as "Max entries",
    MIN(num_entries) as "Min entries"
FROM(
        SELECT player_api_id,
            count(player_api_id) as num_entries
        FROM Player_Attributes
        GROUP BY player_api_id
    )
    """
).to_df()

,Max entries,Min entries
0,56,2


#### Who's the best player?

In [188]:
con.query(
    """--sql
SELECT *
FROM Player
WHERE player_api_id = (
        SELECT DISTINCT player_api_id
        FROM Player_Attributes
        WHERE overall_rating = (
                SELECT MAX(overall_rating)
                FROM Player_Attributes
            )
    ) """
).to_df()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,6176,30981,Lionel Messi,158023,1987-06-24 00:00:00,170.18,159


### Teams

#### What are the contents of Team table?

In [189]:
Markdown(tabulate(con.query(
    """--sql
SELECT * from Team LIMIT 5"""
).to_df(), showindex=False, headers="keys", tablefmt="pipe"))

|   id |   team_api_id |   team_fifa_api_id | team_long_name    | team_short_name   |
|-----:|--------------:|-------------------:|:------------------|:------------------|
|    1 |          9987 |                673 | KRC Genk          | GEN               |
|    2 |          9993 |                675 | Beerschot AC      | BAC               |
|    3 |         10000 |              15005 | SV Zulte-Waregem  | ZUL               |
|    4 |          9994 |               2007 | Sporting Lokeren  | LOK               |
|    5 |          9984 |               1750 | KSV Cercle Brugge | CEB               |

#### Are there null values in the Teams table?

In [190]:
dbf.check_db_nulls(con, 'Team')

No nulls in id Column

No nulls in team_api_id Column

No nulls in team_fifa_api_id Column

No nulls in team_long_name Column

No nulls in team_short_name Column

11 teams do not have team_fifa_api_id.

#### How many different teams are there?

In [191]:
con.query(
    f"""--sql
SELECT count(DISTINCT team_api_id) from Team"""
).fetchone()[0]

299

#### What are the contents of Team_Attributes table?

In [192]:
Markdown(tabulate(con.query(
    f"""--sql
SELECT * from Team_Attributes LIMIT 5"""
).to_df(), showindex=False, headers="keys", tablefmt="pipe"))

|   id |   team_fifa_api_id |   team_api_id | date                |   buildUpPlaySpeed | buildUpPlaySpeedClass   |   buildUpPlayDribbling | buildUpPlayDribblingClass   |   buildUpPlayPassing | buildUpPlayPassingClass   | buildUpPlayPositioningClass   |   chanceCreationPassing | chanceCreationPassingClass   |   chanceCreationCrossing | chanceCreationCrossingClass   |   chanceCreationShooting | chanceCreationShootingClass   | chanceCreationPositioningClass   |   defencePressure | defencePressureClass   |   defenceAggression | defenceAggressionClass   |   defenceTeamWidth | defenceTeamWidthClass   | defenceDefenderLineClass   |
|-----:|-------------------:|--------------:|:--------------------|-------------------:|:------------------------|-----------------------:|:----------------------------|---------------------:|:--------------------------|:------------------------------|------------------------:|:-----------------------------|-------------------------:|:------------------------------|-------------------------:|:------------------------------|:---------------------------------|------------------:|:-----------------------|--------------------:|:-------------------------|-------------------:|:------------------------|:---------------------------|
|    1 |                434 |          9930 | 2010-02-22 00:00:00 |                 60 | Balanced                |                    nan | Little                      |                   50 | Mixed                     | Organised                     |                      60 | Normal                       |                       65 | Normal                        |                       55 | Normal                        | Organised                        |                50 | Medium                 |                  55 | Press                    |                 45 | Normal                  | Cover                      |
|    2 |                434 |          9930 | 2014-09-19 00:00:00 |                 52 | Balanced                |                     48 | Normal                      |                   56 | Mixed                     | Organised                     |                      54 | Normal                       |                       63 | Normal                        |                       64 | Normal                        | Organised                        |                47 | Medium                 |                  44 | Press                    |                 54 | Normal                  | Cover                      |
|    3 |                434 |          9930 | 2015-09-10 00:00:00 |                 47 | Balanced                |                     41 | Normal                      |                   54 | Mixed                     | Organised                     |                      54 | Normal                       |                       63 | Normal                        |                       64 | Normal                        | Organised                        |                47 | Medium                 |                  44 | Press                    |                 54 | Normal                  | Cover                      |
|    4 |                 77 |          8485 | 2010-02-22 00:00:00 |                 70 | Fast                    |                    nan | Little                      |                   70 | Long                      | Organised                     |                      70 | Risky                        |                       70 | Lots                          |                       70 | Lots                          | Organised                        |                60 | Medium                 |                  70 | Double                   |                 70 | Wide                    | Cover                      |
|    5 |                 77 |          8485 | 2011-02-22 00:00:00 |                 47 | Balanced                |                    nan | Little                      |                   52 | Mixed                     | Organised                     |                      53 | Normal                       |                       48 | Normal                        |                       52 | Normal                        | Organised                        |                47 | Medium                 |                  47 | Press                    |                 52 | Normal                  | Cover                      |

#### How many different teams have entries in the Team_Attributes table?

##### by team_api_id:

In [193]:
con.query(
    f"""--sql
SELECT count(distinct team_api_id) from Team_Attributes"""
).fetchone()[0]

288

##### by team_fifa_api_id:

In [194]:
con.query(
    f"""--sql
SELECT count(distinct team_fifa_api_id) from Team_Attributes"""
).fetchone()[0]

285

#### Do all teams from "Team" table have entries in Team_Attributes?

In [195]:
dbf.check_db_refs(con, "Team", "Team_Attributes", "team_api_id")

11 team_api_id values from Team were found to be unreferenced in Team_Attributes:

|    id |   team_api_id |   team_fifa_api_id | team_long_name               | team_short_name   |
|------:|--------------:|-------------------:|:-----------------------------|:------------------|
|     9 |          7947 |                nan | FCV Dender EH                | DEN               |
|    15 |          4049 |                nan | Tubize                       | TUB               |
| 26561 |          6601 |                nan | FC Volendam                  | VOL               |
| 34816 |        177361 |                nan | Termalica Bruk-Bet Nieciecza | TBN               |
| 35286 |          7992 |                nan | Trofense                     | TRO               |

It was the same teams that did not have a team_fifa_api_id

#### Are there missing values in Team_Attributes?

In [196]:
dbf.check_db_nulls(con, 'Team_Attributes')

No nulls in id Column

No nulls in team_fifa_api_id Column

No nulls in team_api_id Column

No nulls in date Column

No nulls in buildUpPlaySpeed Column

...

No nulls in defenceAggression Column

No nulls in defenceAggressionClass Column

No nulls in defenceTeamWidth Column

No nulls in defenceTeamWidthClass Column

No nulls in defenceDefenderLineClass Column

A substantial amount of teams do not have the buildUpPlayDribbling attribute.

### Match

#### What are the contents of the Match table?

In [197]:
Markdown(tabulate(con.query(
    f"""--sql
SELECT * from Match LIMIT 5"""
).to_df(), showindex=False, headers="keys", tablefmt="pipe"))

|   id |   country_id |   league_id | season    |   stage | date                |   match_api_id |   home_team_api_id |   away_team_api_id |   home_team_goal |   away_team_goal |   home_player_X1 |   home_player_X2 |   home_player_X3 |   home_player_X4 |   home_player_X5 |   home_player_X6 |   home_player_X7 |   home_player_X8 |   home_player_X9 |   home_player_X10 |   home_player_X11 |   away_player_X1 |   away_player_X2 |   away_player_X3 |   away_player_X4 |   away_player_X5 |   away_player_X6 |   away_player_X7 |   away_player_X8 |   away_player_X9 |   away_player_X10 |   away_player_X11 |   home_player_Y1 |   home_player_Y2 |   home_player_Y3 |   home_player_Y4 |   home_player_Y5 |   home_player_Y6 |   home_player_Y7 |   home_player_Y8 |   home_player_Y9 |   home_player_Y10 |   home_player_Y11 |   away_player_Y1 |   away_player_Y2 |   away_player_Y3 |   away_player_Y4 |   away_player_Y5 |   away_player_Y6 |   away_player_Y7 |   away_player_Y8 |   away_player_Y9 |   away_player_Y10 |   away_player_Y11 |   home_player_1 |   home_player_2 |   home_player_3 |   home_player_4 |   home_player_5 |   home_player_6 |   home_player_7 |   home_player_8 |   home_player_9 |   home_player_10 |   home_player_11 |   away_player_1 |   away_player_2 |   away_player_3 |   away_player_4 |   away_player_5 |   away_player_6 |   away_player_7 |   away_player_8 |   away_player_9 |   away_player_10 |   away_player_11 |   goal |   shoton |   shotoff |   foulcommit |   card |   cross |   corner |   possession |   B365H |   B365D |   B365A |   BWH |   BWD |   BWA |   IWH |   IWD |   IWA |   LBH |   LBD |   LBA |   PSH |   PSD |   PSA |   WHH |   WHD |   WHA |   SJH |   SJD |   SJA |   VCH |   VCD |   VCA |   GBH |   GBD |   GBA |   BSH |   BSD |   BSA |
|-----:|-------------:|------------:|:----------|--------:|:--------------------|---------------:|-------------------:|-------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|-----------------:|-----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|-----------------:|-----------------:|-------:|---------:|----------:|-------------:|-------:|--------:|---------:|-------------:|--------:|--------:|--------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|
|    1 |            1 |           1 | 2008/2009 |       1 | 2008-08-17 00:00:00 |         492473 |               9987 |               9993 |                1 |                1 |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |    nan |      nan |       nan |          nan |    nan |     nan |      nan |          nan |    1.73 |    3.4  |    5    |  1.75 |  3.35 |  4.2  |  1.85 |   3.2 |   3.5 |  1.8  |   3.3 |  3.75 |   nan |   nan |   nan |  1.7  |  3.3  |  4.33 |  1.9  |   3.3 |  4    |  1.65 |  3.4  |  4.5  |  1.78 |  3.25 |  4    |  1.73 |  3.4  |  4.2  |
|    2 |            1 |           1 | 2008/2009 |       1 | 2008-08-16 00:00:00 |         492474 |              10000 |               9994 |                0 |                0 |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |    nan |      nan |       nan |          nan |    nan |     nan |      nan |          nan |    1.95 |    3.2  |    3.6  |  1.8  |  3.3  |  3.95 |  1.9  |   3.2 |   3.5 |  1.9  |   3.2 |  3.5  |   nan |   nan |   nan |  1.83 |  3.3  |  3.6  |  1.95 |   3.3 |  3.8  |  2    |  3.25 |  3.25 |  1.85 |  3.25 |  3.75 |  1.91 |  3.25 |  3.6  |
|    3 |            1 |           1 | 2008/2009 |       1 | 2008-08-16 00:00:00 |         492475 |               9984 |               8635 |                0 |                3 |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |    nan |      nan |       nan |          nan |    nan |     nan |      nan |          nan |    2.38 |    3.3  |    2.75 |  2.4  |  3.3  |  2.55 |  2.6  |   3.1 |   2.3 |  2.5  |   3.2 |  2.5  |   nan |   nan |   nan |  2.5  |  3.25 |  2.4  |  2.63 |   3.3 |  2.5  |  2.35 |  3.25 |  2.65 |  2.5  |  3.2  |  2.5  |  2.3  |  3.2  |  2.75 |
|    4 |            1 |           1 | 2008/2009 |       1 | 2008-08-17 00:00:00 |         492476 |               9991 |               9998 |                5 |                0 |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |    nan |      nan |       nan |          nan |    nan |     nan |      nan |          nan |    1.44 |    3.75 |    7.5  |  1.4  |  4    |  6.8  |  1.4  |   3.9 |   6   |  1.44 |   3.6 |  6.5  |   nan |   nan |   nan |  1.44 |  3.75 |  6    |  1.44 |   4   |  7.5  |  1.45 |  3.75 |  6.5  |  1.5  |  3.75 |  5.5  |  1.44 |  3.75 |  6.5  |
|    5 |            1 |           1 | 2008/2009 |       1 | 2008-08-16 00:00:00 |         492477 |               7947 |               9985 |                1 |                3 |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |    nan |      nan |       nan |          nan |    nan |     nan |      nan |          nan |    5    |    3.5  |    1.65 |  5    |  3.5  |  1.6  |  4    |   3.3 |   1.7 |  4    |   3.4 |  1.72 |   nan |   nan |   nan |  4.2  |  3.4  |  1.7  |  4.5  |   3.5 |  1.73 |  4.5  |  3.4  |  1.65 |  4.5  |  3.5  |  1.65 |  4.75 |  3.3  |  1.67 |

#### Are all team participants referenced in the Teams table?

##### Home teams:

In [198]:
dbf.check_db_refs(con, 'Match','Team','home_team_api_id','team_api_id')

All entries in Match are referenced in Team.

##### Away teams:

In [199]:
dbf.check_db_refs(con, 'Match','Team','away_team_api_id','team_api_id')

All entries in Match are referenced in Team.

#### What are the missing values?

In [205]:
dbf.check_db_nulls(con, 'Match',row_limit=1)

No nulls in id Column

No nulls in country_id Column

No nulls in league_id Column

No nulls in season Column

No nulls in stage Column

...

|   id |   country_id |   league_id | season    |   stage | date                |   match_api_id |   home_team_api_id |   away_team_api_id |   home_team_goal |   away_team_goal |   home_player_X1 |   home_player_X2 |   home_player_X3 |   home_player_X4 |   home_player_X5 |   home_player_X6 |   home_player_X7 |   home_player_X8 |   home_player_X9 |   home_player_X10 |   home_player_X11 |   away_player_X1 |   away_player_X2 |   away_player_X3 |   away_player_X4 |   away_player_X5 |   away_player_X6 |   away_player_X7 |   away_player_X8 |   away_player_X9 |   away_player_X10 |   away_player_X11 |   home_player_Y1 |   home_player_Y2 |   home_player_Y3 |   home_player_Y4 |   home_player_Y5 |   home_player_Y6 |   home_player_Y7 |   home_player_Y8 |   home_player_Y9 |   home_player_Y10 |   home_player_Y11 |   away_player_Y1 |   away_player_Y2 |   away_player_Y3 |   away_player_Y4 |   away_player_Y5 |   away_player_Y6 |   away_player_Y7 |   away_player_Y8 |   away_player_Y9 |   away_player_Y10 |   away_player_Y11 |   home_player_1 |   home_player_2 |   home_player_3 |   home_player_4 |   home_player_5 |   home_player_6 |   home_player_7 |   home_player_8 |   home_player_9 |   home_player_10 |   home_player_11 |   away_player_1 |   away_player_2 |   away_player_3 |   away_player_4 |   away_player_5 |   away_player_6 |   away_player_7 |   away_player_8 |   away_player_9 |   away_player_10 |   away_player_11 |   goal |   shoton |   shotoff |   foulcommit |   card |   cross |   corner |   possession |   B365H |   B365D |   B365A |   BWH |   BWD |   BWA |   IWH |   IWD |   IWA |   LBH |   LBD |   LBA |   PSH |   PSD |   PSA |   WHH |   WHD |   WHA |   SJH |   SJD |   SJA |   VCH |   VCD |   VCA |   GBH |   GBD |   GBA |   BSH |   BSD |   BSA |
|-----:|-------------:|------------:|:----------|--------:|:--------------------|---------------:|-------------------:|-------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|-----------------:|-----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|-----------------:|-----------------:|-------:|---------:|----------:|-------------:|-------:|--------:|---------:|-------------:|--------:|--------:|--------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|
|    6 |            1 |           1 | 2008/2009 |       1 | 2008-09-24 00:00:00 |         492478 |               8203 |               8342 |                1 |                1 |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |    nan |      nan |       nan |          nan |    nan |     nan |      nan |          nan |    4.75 |     3.4 |    1.67 |  4.85 |   3.4 |  1.65 |   3.7 |   3.2 |   1.8 |     5 |  3.25 |  1.62 |   nan |   nan |   nan |   4.2 |   3.4 |   1.7 |   5.5 |  3.75 |  1.67 |  4.35 |   3.4 |   1.7 |   4.5 |   3.4 |   1.7 |   nan |   nan |   nan |

11818 null rows found in BSD:

|   id |   country_id |   league_id | season    |   stage | date                |   match_api_id |   home_team_api_id |   away_team_api_id |   home_team_goal |   away_team_goal |   home_player_X1 |   home_player_X2 |   home_player_X3 |   home_player_X4 |   home_player_X5 |   home_player_X6 |   home_player_X7 |   home_player_X8 |   home_player_X9 |   home_player_X10 |   home_player_X11 |   away_player_X1 |   away_player_X2 |   away_player_X3 |   away_player_X4 |   away_player_X5 |   away_player_X6 |   away_player_X7 |   away_player_X8 |   away_player_X9 |   away_player_X10 |   away_player_X11 |   home_player_Y1 |   home_player_Y2 |   home_player_Y3 |   home_player_Y4 |   home_player_Y5 |   home_player_Y6 |   home_player_Y7 |   home_player_Y8 |   home_player_Y9 |   home_player_Y10 |   home_player_Y11 |   away_player_Y1 |   away_player_Y2 |   away_player_Y3 |   away_player_Y4 |   away_player_Y5 |   away_player_Y6 |   away_player_Y7 |   away_player_Y8 |   away_player_Y9 |   away_player_Y10 |   away_player_Y11 |   home_player_1 |   home_player_2 |   home_player_3 |   home_player_4 |   home_player_5 |   home_player_6 |   home_player_7 |   home_player_8 |   home_player_9 |   home_player_10 |   home_player_11 |   away_player_1 |   away_player_2 |   away_player_3 |   away_player_4 |   away_player_5 |   away_player_6 |   away_player_7 |   away_player_8 |   away_player_9 |   away_player_10 |   away_player_11 |   goal |   shoton |   shotoff |   foulcommit |   card |   cross |   corner |   possession |   B365H |   B365D |   B365A |   BWH |   BWD |   BWA |   IWH |   IWD |   IWA |   LBH |   LBD |   LBA |   PSH |   PSD |   PSA |   WHH |   WHD |   WHA |   SJH |   SJD |   SJA |   VCH |   VCD |   VCA |   GBH |   GBD |   GBA |   BSH |   BSD |   BSA |
|-----:|-------------:|------------:|:----------|--------:|:--------------------|---------------:|-------------------:|-------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|-----------------:|-----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|-----------------:|-----------------:|-------:|---------:|----------:|-------------:|-------:|--------:|---------:|-------------:|--------:|--------:|--------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|
|    6 |            1 |           1 | 2008/2009 |       1 | 2008-09-24 00:00:00 |         492478 |               8203 |               8342 |                1 |                1 |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |    nan |      nan |       nan |          nan |    nan |     nan |      nan |          nan |    4.75 |     3.4 |    1.67 |  4.85 |   3.4 |  1.65 |   3.7 |   3.2 |   1.8 |     5 |  3.25 |  1.62 |   nan |   nan |   nan |   4.2 |   3.4 |   1.7 |   5.5 |  3.75 |  1.67 |  4.35 |   3.4 |   1.7 |   4.5 |   3.4 |   1.7 |   nan |   nan |   nan |

11818 null rows found in BSA:

|   id |   country_id |   league_id | season    |   stage | date                |   match_api_id |   home_team_api_id |   away_team_api_id |   home_team_goal |   away_team_goal |   home_player_X1 |   home_player_X2 |   home_player_X3 |   home_player_X4 |   home_player_X5 |   home_player_X6 |   home_player_X7 |   home_player_X8 |   home_player_X9 |   home_player_X10 |   home_player_X11 |   away_player_X1 |   away_player_X2 |   away_player_X3 |   away_player_X4 |   away_player_X5 |   away_player_X6 |   away_player_X7 |   away_player_X8 |   away_player_X9 |   away_player_X10 |   away_player_X11 |   home_player_Y1 |   home_player_Y2 |   home_player_Y3 |   home_player_Y4 |   home_player_Y5 |   home_player_Y6 |   home_player_Y7 |   home_player_Y8 |   home_player_Y9 |   home_player_Y10 |   home_player_Y11 |   away_player_Y1 |   away_player_Y2 |   away_player_Y3 |   away_player_Y4 |   away_player_Y5 |   away_player_Y6 |   away_player_Y7 |   away_player_Y8 |   away_player_Y9 |   away_player_Y10 |   away_player_Y11 |   home_player_1 |   home_player_2 |   home_player_3 |   home_player_4 |   home_player_5 |   home_player_6 |   home_player_7 |   home_player_8 |   home_player_9 |   home_player_10 |   home_player_11 |   away_player_1 |   away_player_2 |   away_player_3 |   away_player_4 |   away_player_5 |   away_player_6 |   away_player_7 |   away_player_8 |   away_player_9 |   away_player_10 |   away_player_11 |   goal |   shoton |   shotoff |   foulcommit |   card |   cross |   corner |   possession |   B365H |   B365D |   B365A |   BWH |   BWD |   BWA |   IWH |   IWD |   IWA |   LBH |   LBD |   LBA |   PSH |   PSD |   PSA |   WHH |   WHD |   WHA |   SJH |   SJD |   SJA |   VCH |   VCD |   VCA |   GBH |   GBD |   GBA |   BSH |   BSD |   BSA |
|-----:|-------------:|------------:|:----------|--------:|:--------------------|---------------:|-------------------:|-------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|-----------------:|------------------:|------------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|-----------------:|-----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|----------------:|-----------------:|-----------------:|-------:|---------:|----------:|-------------:|-------:|--------:|---------:|-------------:|--------:|--------:|--------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|
|    6 |            1 |           1 | 2008/2009 |       1 | 2008-09-24 00:00:00 |         492478 |               8203 |               8342 |                1 |                1 |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |              nan |               nan |               nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |             nan |              nan |              nan |    nan |      nan |       nan |          nan |    nan |     nan |      nan |          nan |    4.75 |     3.4 |    1.67 |  4.85 |   3.4 |  1.65 |   3.7 |   3.2 |   1.8 |     5 |  3.25 |  1.62 |   nan |   nan |   nan |   4.2 |   3.4 |   1.7 |   5.5 |  3.75 |  1.67 |  4.35 |   3.4 |   1.7 |   4.5 |   3.4 |   1.7 |   nan |   nan |   nan |

There are several thousand entries that are missing team members or their positions, there are more than ten thousand entries missing some match statistics or betting coefficients.

In [201]:
Markdown(tabulate(con.query(
    f"""--sql
SELECT * from League LIMIT 5"""
).to_df(), showindex=False, headers="keys", tablefmt="pipe"))

|    id |   country_id | name                   |
|------:|-------------:|:-----------------------|
|     1 |            1 | Belgium Jupiler League |
|  1729 |         1729 | England Premier League |
|  4769 |         4769 | France Ligue 1         |
|  7809 |         7809 | Germany 1. Bundesliga  |
| 10257 |        10257 | Italy Serie A          |

In [202]:
Markdown(tabulate(con.query(
    f"""--sql
SELECT league_id, count(id) from Match group by league_id"""
).to_df(), showindex=False, headers="keys", tablefmt="pipe"))

|   league_id |   count(id) |
|------------:|------------:|
|           1 |        1728 |
|        1729 |        3040 |
|        4769 |        3040 |
|        7809 |        2448 |
|       10257 |        3017 |
|       13274 |        2448 |
|       15722 |        1920 |
|       17642 |        2052 |
|       19694 |        1824 |
|       21518 |        3040 |
|       24558 |        1422 |